In [29]:
import numpy as np
import pynn

In [30]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

def load_minibatches(batch_size=64):
    tsfms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    trn_set = datasets.MNIST('./data/train', train=True, download=True, transform=tsfms)
    trn_loader = DataLoader(trn_set, batch_size=batch_size, shuffle=True, num_workers=0)
    data = []
    for mb in trn_loader:
        inputs_t,targets_t = mb
        inputs = np.zeros((inputs_t.size(0),784))
        targets = np.zeros((inputs_t.size(0),10))
        for i in range(0,inputs_t.size(0)):
            targets[i,targets_t[i]] = 1.
            for j in range(0,28):
                for k in range(0,28):
                    inputs[i,j*28+k] = inputs_t[i,0,j,k]
        data.append((inputs,targets))
    return data

In [31]:
nn = pynn.NeuralNetwork(name='Test NN')
l1 = pynn.Linear(in_features=784, out_features=128, bias=True, initialization='random', name='Linear 1')
a1 = pynn.ReLU(name='Relu 1')
l2 = pynn.Linear(in_features=128, out_features=10, bias=True, initialization='random', name='Linear 2')
a2 = pynn.Softmax(name='Softmax')

nn.add(block_name='input', layer=l1)
nn.add(block_name='input', layer=a1)
nn.add(block_name='output', layer=l2)
nn.add(block_name='output', layer=a2)
print(nn.summary())

Model Summary - Test NN:
-------------------------
Input - 
Linear Layer: 784 -> 128
ReLU Layer: Relu 1
Hidden - 
Output - 
Linear Layer: 128 -> 10
Softmax Layer: Softmax
-------------------------



In [36]:
def SGD(model, loss, lr, epochs, data):
    for epoch in range(epochs):
        running_loss = 0.
        batches_seen = 0
        for minibatch_x, minibatch_y in data:
            batches_seen += minibatch_x.shape[0]
            minibatch_y_hat = model(minibatch_x)
            running_loss += loss(minibatch_y_hat, minibatch_y).sum()
            model.backward(loss.backward())
            model.update_weights(lr)
        print(f'Epoch {epoch + 1} / {epochs} : Loss = {running_loss / batches_seen}')

In [37]:
batch_size = 128
lr = 0.001
input_size = (1, 28, 28)
epochs = 10

In [12]:
data = load_minibatches(batch_size=batch_size)

In [38]:
loss = pynn.CrossEntropy()

In [39]:
SGD(model=nn, loss=loss, lr=lr, epochs=epochs, data=data)

Epoch 1 / 10 : Loss = 1.5620225674789427
Epoch 2 / 10 : Loss = 0.803754901210988
Epoch 3 / 10 : Loss = 0.606792314027842


KeyboardInterrupt: 